# Workflow

In [1]:
# Import dependencies
import pandas as pd
from pathlib import Path

from datetime import datetime
import matplotlib.pyplot as plt

## Import Datasets

In [2]:
# Import latest dataset
current_path = Path("resources/202310-citibike-tripdata.csv")
current_df = pd.read_csv(
    current_path,
    dtype = {
        'start_station_id': str, # dtype warning, column 5
        'end_station_id': str # dtype warning, column 7
    })

In [3]:
# Import previous dataset
prev_path = Path("resources/202309-citibike-tripdata.csv")
prev_df = pd.read_csv(
    prev_path,
    dtype = {
        'start_station_id': str, # dtype warning, column 5
        'end_station_id': str # dtype warning, column 7
    })

In [4]:
# Display the current_df
current_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,F01D2D54E9E60D6E,classic_bike,2023-10-03 02:48:38,2023-10-03 02:48:40,Columbus Pl & Atlantic Ave,4146.02,Columbus Pl & Atlantic Ave,4146.02,40.677223,-73.922792,40.677170,-73.922850,casual
1,CAE4EDBEA07001BD,classic_bike,2023-10-11 16:03:17,2023-10-11 16:45:26,Central Park West & W 85 St,7354.01,Central Park West & W 85 St,7354.01,40.784760,-73.969862,40.784760,-73.969862,casual
2,FDC34BAD31193E07,classic_bike,2023-10-11 19:57:13,2023-10-11 20:20:10,Hicks St & Montague St,4645.09,5 St & 6 Ave,3874.01,40.694974,-73.995936,40.670484,-73.982090,casual
3,DFEA5E65AE91CE2A,classic_bike,2023-10-10 20:18:22,2023-10-10 20:18:37,Atlantic Ave & Furman St,4614.04,Atlantic Ave & Furman St,4614.04,40.691669,-74.000139,40.691652,-73.999979,casual
4,48299D8BE9B55255,classic_bike,2023-10-17 16:26:58,2023-10-17 16:34:27,E 41 St & Madison Ave (SE corner),6432.10,E 58 St & 3 Ave,6762.02,40.751845,-73.979585,40.760958,-73.967245,casual


In [5]:
# Display the prev_df
prev_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,B0A0F1DEFA4B72FC,electric_bike,2023-09-03 10:20:41,2023-09-03 10:24:16,E 1 St & Bowery,5636.13,E 10 St & 2 Ave,5746.02,40.724861,-73.992131,40.729708,-73.986598,member
1,2B26AB15647BF4EE,classic_bike,2023-09-27 15:44:23,2023-09-27 15:53:25,Pearl St & Hanover Square,4993.02,Allen St & Rivington St,5414.06,40.704650,-74.009133,40.720196,-73.989978,member
2,9D2B5971CA4E513F,classic_bike,2023-09-19 13:40:48,2023-09-19 13:48:11,E 1 St & Bowery,5636.13,E 10 St & 2 Ave,5746.02,40.724753,-73.992116,40.729708,-73.986598,member
3,17E6760596DC3ABE,classic_bike,2023-09-30 16:27:50,2023-09-30 16:56:35,Central Ave & Himrod St,4713.01,Mott St & Prince St,5561.04,40.696706,-73.922935,40.723180,-73.994800,member
4,97EFF376A7E2DC70,classic_bike,2023-09-21 16:59:53,2023-09-21 17:07:36,St Marks Pl & 2 Ave,5669.10,Mott St & Prince St,5561.04,40.728419,-73.987140,40.723180,-73.994800,member


In [6]:
# Create an array of the dataframes
datasets = [current_df, prev_df]

## Data Cleaning

In [7]:
# Define the columns to drop
drop_columns = ['ride_id', 'start_station_id', 'end_station_id']

# Initialise array to hold the output
clean_datasets = []

# Loop through each dataframe
for df in datasets:
    
    # Drop rows with null values
    drop_nulls = df.dropna(how="any").reset_index(drop=True)
    
    # Drop columns: `ride_id`, `start_station_id`, `end_station_id`
    reduced_df = drop_nulls.drop(columns=drop_columns)
    
    # Convert the `started_at` and `ended_at` columns to datetime
    clean_df = reduced_df.astype({
        'started_at': 'datetime64[ns]',
        'ended_at': 'datetime64[ns]'
    })
    
    # Calculate the trip duration
    duration = clean_df.ended_at - clean_df.started_at
    
    # Drop rows with a negative duration
    positive_durations = clean_df.loc[duration > pd.Timedelta(0)].reset_index(drop=True).copy()
    
    # Append to a new list
    clean_datasets.append(positive_durations)
    
    # Display the info
    positive_durations.info(show_counts=True)
    print()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3805477 entries, 0 to 3805476
Data columns (total 10 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   rideable_type       3805477 non-null  object        
 1   started_at          3805477 non-null  datetime64[ns]
 2   ended_at            3805477 non-null  datetime64[ns]
 3   start_station_name  3805477 non-null  object        
 4   end_station_name    3805477 non-null  object        
 5   start_lat           3805477 non-null  float64       
 6   start_lng           3805477 non-null  float64       
 7   end_lat             3805477 non-null  float64       
 8   end_lng             3805477 non-null  float64       
 9   member_casual       3805477 non-null  object        
dtypes: datetime64[ns](2), float64(4), object(4)
memory usage: 290.3+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3558894 entries, 0 to 3558893
Data columns (total 10 columns):
 #  

### Duration (mins) column

In [8]:
# Function to find the upper bounds
def find_upper_bounds(check_column):
    # Calculate the IQR, lower/upper bounds
    lower_quartile = check_column['25%']
    upper_quartile = check_column['75%']

    # Calculate the IQR
    iqr = upper_quartile - lower_quartile

    # Calculate the upper bounds
    upper_bounds = upper_quartile + (1.5*iqr)
    
    return(upper_bounds)

In [9]:
# Loop through each dataframe
for df in clean_datasets:
    
    # Add the new column
    df['duration_in_mins'] = pd.Series(dtype=int)
    
    # Calculate the updated duration
    duration = df.ended_at - df.started_at
    
    # Convert to minutes, as integer
    duration_in_mins = (duration.dt.total_seconds() / 60).astype(int)

    # Descriptive statistics, as integers
    duration_stats = duration_in_mins.describe().apply(lambda x: round(x))
    print(duration_stats)
    
    # Add the duration_in_mins as a new column
    df['duration_in_mins'] = duration_in_mins
    
    ######## LONG TRIPS ########
    print(f"Upper bounds: {find_upper_bounds(duration_stats)}")
    print()

count    3805477
mean          13
std           29
min            0
25%            5
50%            9
75%           16
max         1499
dtype: int64
Upper bounds: 32.5

count    3558894
mean          14
std           32
min            0
25%            5
50%            9
75%           16
max         1498
dtype: int64
Upper bounds: 32.5



### Trip Validity - Short vs Long

In [10]:
# Identify the "short" trips
for df in clean_datasets:
    # Add the new column
    df['trip_validity'] = pd.Series(dtype=str)
    
    # Calculate the updated duration
    duration = df.ended_at - df.started_at
    
    # Identify trips < 1 minute (potential user error, incorrect docking, etc.)
    lessthan_minute = df.loc[duration < pd.Timedelta(minutes=1)]
    
    # Determine trips which occurred at the same location
    same_location = lessthan_minute.loc[lessthan_minute.start_station_name == lessthan_minute.end_station_name]
    
    # Update the `trip_validity` column
    df.loc[same_location.index, 'trip_validity'] = "short"

In [11]:
# Identify the "long" trips, by membership type
for df in clean_datasets:
    ######## CASUAL ########
    casual_trips = (df.member_casual == "casual")
    casual_limit = (duration > pd.Timedelta(minutes=30))
    
    # Calculate the upper bounds for casual trips exceeding 30 minutes
    exceeded_casual = df.loc[casual_trips & casual_limit]
    casual_upper = find_upper_bounds(exceeded_casual.duration_in_mins.describe())
    print(f"Casual - Upper bounds: {casual_upper}")
    
    # Identify rows of outliers
    casual_outliers = exceeded_casual.loc[duration > pd.Timedelta(minutes=casual_upper)]
    
    # Update the `trip_validity` column
    df.loc[casual_outliers.index, 'trip_validity'] = "long"
    
    ######## MEMBER ########
    member_trips = (df.member_casual == "member")
    member_limit = (duration > pd.Timedelta(minutes=45))
    
    # Calculate the upper bounds for member trips exceeding 45 minutes
    exceeded_member = df.loc[member_trips & member_limit]
    member_upper = find_upper_bounds(exceeded_member.duration_in_mins.describe())
    print(f"Member - Upper bounds: {member_upper}\n")
    
    # Identify rows of outliers
    member_outliers = exceeded_member.loc[duration > pd.Timedelta(minutes=member_upper)]
    
    # Update the `trip_validity` column
    df.loc[member_outliers.index, 'trip_validity'] = "long"
    
    ######## VALID TRIPS ########
    # Fill NaN trips in `trip_validity` with "valid"
    df.trip_validity.fillna(value="valid", inplace=True)

Casual - Upper bounds: 44.5
Member - Upper bounds: 29.0

Casual - Upper bounds: 99.0
Member - Upper bounds: 105.5



## Confirm and Export Datasets

In [12]:
clean_datasets[0].head()

,rideable_type,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual,duration_in_mins,trip_validity
0,classic_bike,2023-10-03 02:48:38,2023-10-03 02:48:40,Columbus Pl & Atlantic Ave,Columbus Pl & Atlantic Ave,40.677223,-73.922792,40.677170,-73.922850,casual,0,short
1,classic_bike,2023-10-11 16:03:17,2023-10-11 16:45:26,Central Park West & W 85 St,Central Park West & W 85 St,40.784760,-73.969862,40.784760,-73.969862,casual,42,valid
2,classic_bike,2023-10-11 19:57:13,2023-10-11 20:20:10,Hicks St & Montague St,5 St & 6 Ave,40.694974,-73.995936,40.670484,-73.982090,casual,22,valid
3,classic_bike,2023-10-10 20:18:22,2023-10-10 20:18:37,Atlantic Ave & Furman St,Atlantic Ave & Furman St,40.691669,-74.000139,40.691652,-73.999979,casual,0,short
4,classic_bike,2023-10-17 16:26:58,2023-10-17 16:34:27,E 41 St & Madison Ave (SE corner),E 58 St & 3 Ave,40.751845,-73.979585,40.760958,-73.967245,casual,7,valid


In [20]:
clean_datasets[0].end_station_name.value_counts()

end_station_name
W 21 St & 6 Ave                      14670
West St & Chambers St                13441
University Pl & E 14 St              12039
E 41 St & Madison Ave (SE corner)    11914
Broadway & W 58 St                   11609
                                     ...  
Sedgwick Ave & Hall of Fame Tce          1
Madison St & 1 St                        1
Journal Square                           1
Bergen Ave                               1
River St & 1 St                          1
Name: count, Length: 2151, dtype: int64

In [13]:
clean_datasets[1].head()

,rideable_type,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual,duration_in_mins,trip_validity
0,electric_bike,2023-09-03 10:20:41,2023-09-03 10:24:16,E 1 St & Bowery,E 10 St & 2 Ave,40.724861,-73.992131,40.729708,-73.986598,member,3,valid
1,classic_bike,2023-09-27 15:44:23,2023-09-27 15:53:25,Pearl St & Hanover Square,Allen St & Rivington St,40.704650,-74.009133,40.720196,-73.989978,member,9,valid
2,classic_bike,2023-09-19 13:40:48,2023-09-19 13:48:11,E 1 St & Bowery,E 10 St & 2 Ave,40.724753,-73.992116,40.729708,-73.986598,member,7,valid
3,classic_bike,2023-09-30 16:27:50,2023-09-30 16:56:35,Central Ave & Himrod St,Mott St & Prince St,40.696706,-73.922935,40.723180,-73.994800,member,28,valid
4,classic_bike,2023-09-21 16:59:53,2023-09-21 17:07:36,St Marks Pl & 2 Ave,Mott St & Prince St,40.728419,-73.987140,40.723180,-73.994800,member,7,valid


In [15]:
combined_df = pd.concat([clean_datasets[0], clean_datasets[1]])

In [18]:
combined_df.head()

,rideable_type,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual,duration_in_mins,trip_validity
0,classic_bike,2023-10-03 02:48:38,2023-10-03 02:48:40,Columbus Pl & Atlantic Ave,Columbus Pl & Atlantic Ave,40.677223,-73.922792,40.677170,-73.922850,casual,0,short
1,classic_bike,2023-10-11 16:03:17,2023-10-11 16:45:26,Central Park West & W 85 St,Central Park West & W 85 St,40.784760,-73.969862,40.784760,-73.969862,casual,42,valid
2,classic_bike,2023-10-11 19:57:13,2023-10-11 20:20:10,Hicks St & Montague St,5 St & 6 Ave,40.694974,-73.995936,40.670484,-73.982090,casual,22,valid
3,classic_bike,2023-10-10 20:18:22,2023-10-10 20:18:37,Atlantic Ave & Furman St,Atlantic Ave & Furman St,40.691669,-74.000139,40.691652,-73.999979,casual,0,short
4,classic_bike,2023-10-17 16:26:58,2023-10-17 16:34:27,E 41 St & Madison Ave (SE corner),E 58 St & 3 Ave,40.751845,-73.979585,40.760958,-73.967245,casual,7,valid


In [16]:
combined_df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 7364371 entries, 0 to 3558893
Data columns (total 12 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   rideable_type       7364371 non-null  object        
 1   started_at          7364371 non-null  datetime64[ns]
 2   ended_at            7364371 non-null  datetime64[ns]
 3   start_station_name  7364371 non-null  object        
 4   end_station_name    7364371 non-null  object        
 5   start_lat           7364371 non-null  float64       
 6   start_lng           7364371 non-null  float64       
 7   end_lat             7364371 non-null  float64       
 8   end_lng             7364371 non-null  float64       
 9   member_casual       7364371 non-null  object        
 10  duration_in_mins    7364371 non-null  int64         
 11  trip_validity       7364371 non-null  object        
dtypes: datetime64[ns](2), float64(4), int64(1), object(5)
memory usage: 730.4+ 

In [17]:
combined_df.to_csv("output/workflow/combined_tripdata.csv")